In [1]:
import os
from dotenv import load_dotenv

from src.agents.community_summarizer import CommunitiesSummarizer
from src.config import Source, ChunkerConf, LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.factory.embeddings import get_embeddings
from src.graph.knowledge_graph import KnowledgeGraph

import networkx as nx

env = load_dotenv('config.env')

In [2]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='vector'
)

embedder_conf = EmbedderConf(
    type=os.getenv("EMBEDDINGS_TYPE"),
    model=os.getenv("EMBEDDINGS_MODEL_NAME"),
)

model_conf=LLMConf(
    type=os.getenv("RE_MODEL_TYPE"),
    model=os.getenv("RE_MODEL_NAME"), 
    temperature=os.getenv("RE_MODEL_TEMPERATURE"), 
    deployment=os.getenv("RE_MODEL_DEPLOYMENT"),
    api_key=os.getenv("RE_API_KEY"),
    endpoint=os.getenv("RE_MODEL_ENDPOINT"),
    api_version=os.getenv("RE_MODEL_API_VERSION") or None
)

embeddings = get_embeddings(conf=embedder_conf)

kg = KnowledgeGraph(conf=kg_config,embeddings_model=embeddings)

In [3]:
communities = kg.get_communities()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: " \n            MATCH (n)-[r]-(m)  \n            WHERE n.community_leiden IS NOT NULL\n            OPTIONAL MATCH (chunk:Chunk) WHERE chunk.community_leiden = n.community_leiden  \n            WITH \n                'leiden' AS community_type, \n                n.community_leiden AS community_id, \n                count(DISTINCT n) AS community_size, \n                collect(DISTINCT elementId(n)) AS entity_ids,\n                collect(DISTINCT n.name) AS names,\n                collect(DISTINCT elementId(r)) AS relationship_ids,  \n                collect(DISTINCT type(r)) AS relationship_types,\n                collect(DISTINCT elementId(chunk)) AS chunk_ids    \n            RETUR

In [4]:
communities

[Community(community_type='leiden', community_id=0, community_size=25, entity_ids=['4:44890bb9-4945-4a7c-8f87-381effaa2643:10', '4:44890bb9-4945-4a7c-8f87-381effaa2643:19', '4:44890bb9-4945-4a7c-8f87-381effaa2643:20', '4:44890bb9-4945-4a7c-8f87-381effaa2643:21', '4:44890bb9-4945-4a7c-8f87-381effaa2643:22', '4:44890bb9-4945-4a7c-8f87-381effaa2643:23', '4:44890bb9-4945-4a7c-8f87-381effaa2643:24', '4:44890bb9-4945-4a7c-8f87-381effaa2643:25', '4:44890bb9-4945-4a7c-8f87-381effaa2643:26', '4:44890bb9-4945-4a7c-8f87-381effaa2643:33', '4:44890bb9-4945-4a7c-8f87-381effaa2643:34', '4:44890bb9-4945-4a7c-8f87-381effaa2643:35', '4:44890bb9-4945-4a7c-8f87-381effaa2643:36', '4:44890bb9-4945-4a7c-8f87-381effaa2643:37', '4:44890bb9-4945-4a7c-8f87-381effaa2643:38', '4:44890bb9-4945-4a7c-8f87-381effaa2643:39', '4:44890bb9-4945-4a7c-8f87-381effaa2643:40', '4:44890bb9-4945-4a7c-8f87-381effaa2643:41', '4:44890bb9-4945-4a7c-8f87-381effaa2643:42', '4:44890bb9-4945-4a7c-8f87-381effaa2643:43', '4:44890bb9-4945-

In [5]:
communities[0].chunks

[Chunk(chunk_id='4:44890bb9-4945-4a7c-8f87-381effaa2643:19', text="EU Plan and the 2023 Renewable Energy Directive. It will scale up the European PV manufacturing capacity, develop a more resilient value chain in the EU and reduce dependency on fossil fuels through collaboration across the entire PV value chain. The Commission and the private partners plan to invest up to €240 million each from 2025 to 2030 in the partnership. European Partnership for Textiles of the Future: It will drive the transformation of the textile industry towards sustainability and circularity, aligning with the EU Strategy for Sustainable and Circular Textiles. The initiative will leverage digital innovations and new business models within the sector and enhance Europe's strategic autonomy. At the same time, it will help to keep the industry competitive, resilient, and sustainable in a rapidly changing global market. The Commission and the private partners plan to invest up to €30 million each from 2025 to 20

In [6]:
summarizer = CommunitiesSummarizer(llm_conf=model_conf, embeddings_conf=embedder_conf)


2025-03-31 22:17:59,653 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-03-31 22:17:59,806 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'


In [7]:
summarizer.get_community_report(community=communities[0])

CommunityReport(communtiy_type='leiden', community_id=0, summary='The EU is launching three new partnerships focused on renewable energy, textiles, and advanced materials. These partnerships aim to strengthen European industry, promote sustainability, and reduce reliance on imported technologies. The Commission and private partners will invest a combined €510 million in these initiatives from 2025 to 2030.  \n', rank=0.0, community_size=25, attributes=None, summary_embeddings=[-0.010580387, 0.028421145, -0.034961987, 0.023220856, -0.04567734, 0.03299639, -0.008242998, -0.053491104, 0.02167952, -0.00504785, -0.011273962, 0.0153124435, 0.031564854, -0.015446905, -0.0131695485, 0.011331561, -0.029149741, 0.0068883384, 0.015468597, 0.041338906, -0.009601932, 0.028525991, -0.04921852, -0.050385322, -0.03598527, 0.03906728, 0.011357469, 0.007960342, 0.071910456, 0.040297862, -0.01716525, -0.039372593, 0.008602621, -0.060377505, -0.024330223, -0.0115573825, -0.025020074, 0.051793706, 0.007924

In [8]:
reports = summarizer.get_reports(communities)

reports

[CommunityReport(communtiy_type='leiden', community_id=0, summary='The EU is launching three new partnerships focused on renewable energy, textiles, and advanced materials. These partnerships aim to strengthen European industry, promote sustainability, and reduce reliance on imported technologies. The Commission and private partners will invest a combined €510 million in these initiatives from 2025 to 2030.  \n', rank=0.0, community_size=25, attributes=None, summary_embeddings=[-0.010580387, 0.028421145, -0.034961987, 0.023220856, -0.04567734, 0.03299639, -0.008242998, -0.053491104, 0.02167952, -0.00504785, -0.011273962, 0.0153124435, 0.031564854, -0.015446905, -0.0131695485, 0.011331561, -0.029149741, 0.0068883384, 0.015468597, 0.041338906, -0.009601932, 0.028525991, -0.04921852, -0.050385322, -0.03598527, 0.03906728, 0.011357469, 0.007960342, 0.071910456, 0.040297862, -0.01716525, -0.039372593, 0.008602621, -0.060377505, -0.024330223, -0.0115573825, -0.025020074, 0.051793706, 0.00792

In [9]:
kg.store_community_reports(reports)

### Retrieval of Communities
Now we are able to retrieve from two different vector stores: one dedicated to Chunks (Document vectors) and one for Summaries of Communities.  

Notice that once retrieved, the summary of the community has some metadata attached, such as the `community_id` and the `community_type`, while Chunks metadata could be used to retrieve neighbouring Chunks (`NEXT` relationship) and mentioned Entities (`MENTIONS` relationship).

In [10]:
query = "Why is Europe introducting regulations on Apple?"

kg.cr_store.similarity_search_with_relevance_scores(query, k=2)

[(Document(metadata={'community_type': 'leiden', 'community_id': 4, 'community_size': 11}, page_content="The European Commission issued two decisions requiring Apple to comply with the Digital Markets Act's interoperability obligation. These decisions specify measures for Apple to enable interoperability with iOS for third-party connected devices, aiming to increase consumer choice and foster innovation in the European market.  \n"),
  0.851923942565918),
 (Document(metadata={'community_type': 'leiden', 'community_id': 1, 'community_size': 21}, page_content='The European Commission has issued legally binding decisions requiring Apple to improve interoperability between its iOS and iPadOS operating systems and third-party apps and hardware.  These decisions, based on extensive engagement with Apple and third parties, aim to enhance transparency, fairness, and efficiency in the process for developers seeking interoperability.  Apple must implement these measures within specified timeline

In [11]:
query = "Why is Europe introducting regulations on Apple?"

kg.vector_store.similarity_search_with_relevance_scores(query, k=5)

[(Document(metadata={'document_version': 1, 'closeness': 0.0, 'pagerank': 0.005568836683187605, 'betweenness': 0.0, 'embeddings_model': 'mxbai-embed-large', 'chunk_overlap': 100, 'community_leiden': 4, 'folder': 'source_docs', 'chunk_size': 1000, 'filename': 'Commission_provides_guidance_under_Digital_Markets_Act_to_facilitate_development_of_innovative_products_on_Apple_s_platforms.pdf', 'community_louvain': 2, 'chunk_id': 1}, page_content="European Commission Press release Commission provides guidance under Digital Markets Act to facilitate development of innovative products on Apple's platforms Brussels, 19 March 2025 Today, the European Commission adopted two decisions under the Digital Markets Act (DMA) specifying the measures that Apple has to take to comply with certain aspects of its interoperability obligation. Interoperability enables a deeper and more seamless integration of third-party products with Apple's ecosystem. Interoperability is therefore key to opening up new possi